This code is meant to transform initial file with inventory report into textfile that can be uploaded into GCR.

In [1]:
import pandas as pd
import numpy as np
from datetime import datetime, timedelta, date

Lines below let the user fully observe the initial data and final output.

In [2]:
pd.set_option('display.width', 150)
pd.options.display.max_colwidth = 150

This table shows movement codes meant for movement types.

In [3]:
movement_chart = pd.DataFrame({'Movement_Type': ['Stock','Transit','Shipped','OOS'],
                                'Movement_Code': ['01','04','03','05']},
                              columns=['Movement_Type','Movement_Code'])
print(movement_chart)

  Movement_Type Movement_Code
0         Stock            01
1       Transit            04
2       Shipped            03
3           OOS            05


Excel-file with inventory report is imported and cyrillic column names are changed.

In [4]:
df = pd.read_excel (r'C:\Users\БФедин\OneDrive - GENERIX\InventoryReports\Балтика 20.11-22.11.xlsx')
df = df.rename(columns={'Дата отчета':'Movement_Date','Дата резерва':'Transit_Date', \
                       'РЦ':'Site_Code','Код ТП':'Internal_Code','Наименование ТП':'Item_Name', \
                       'ШК ТП':'EAN_Code','Остаток на РЦ, шт':'Stock','Заказ ТТ, шт':'Demand', \
                       'Отгружено на ТТ, шт':'Shipped','Недогруз в магазины, шт':'OOS','В ожидании поставки, шт':'Transit'})

Blank cells in columns below are replaced with zero.

In [5]:
df[['Stock','Demand','Shipped','OOS','Transit']] = df[['Stock','Demand','Shipped','OOS','Transit']].fillna(0)

Integer format is applied to columns with numbers.

In [6]:
df[['Stock','Demand','Shipped','OOS','Transit']] = df[['Stock','Demand','Shipped','OOS','Transit']].astype(int)

String format is applied to columns with codes. They should be regarded as text values.

In [7]:
df[['Internal_Code','EAN_Code','Site_Code']] = df[['Internal_Code','EAN_Code','Site_Code']].astype(str)

In [8]:
print(df.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 167 entries, 0 to 166
Data columns (total 11 columns):
 #   Column         Non-Null Count  Dtype         
---  ------         --------------  -----         
 0   Movement_Date  167 non-null    datetime64[ns]
 1   Transit_Date   86 non-null     datetime64[ns]
 2   Site_Code      167 non-null    object        
 3   Internal_Code  167 non-null    object        
 4   Item_Name      167 non-null    object        
 5   EAN_Code       167 non-null    object        
 6   Stock          167 non-null    int32         
 7   Demand         167 non-null    int32         
 8   Shipped        167 non-null    int32         
 9   OOS            167 non-null    int32         
 10  Transit        167 non-null    int32         
dtypes: datetime64[ns](2), int32(5), object(4)
memory usage: 11.2+ KB
None


Site Names are replaced with GLN codes.

In [9]:
df['Site_Code'].replace(to_replace='.*[Тт]ольятти.*',value=4680328532412,regex=True, inplace=True )

Case Count is taken from the end of Item's name.

In [10]:
df['Case_Count'] = df['Item_Name'].str.split(':').str[1]

Item Type with 00 means Standard Type. By now it's the only possible value.

In [11]:
df['Item_Type'] = '00'

New format with only date (not time) is applied to column with Transit date.

In [12]:
df['Transit_Date'] = pd.to_datetime(df['Transit_Date'], format='%Y-%m-%d',errors='coerce')

If report is sent on Monday, data for Friday and Saturday may have blank cells in Transit Date columns. For these cases, blank cells are replaced with values from Movement Date column.

In [13]:
df['Transit_Date'] = df['Transit_Date'].fillna(df['Movement_Date'])

Transit with only 4 days after laters Movement Date is kept in dataframe.

In [14]:
end_date = pd.to_datetime(df['Movement_Date'].max()) + pd.DateOffset(days=4)


df['Transit'] = np.where((df['Transit_Date'] > end_date),0,df['Transit'])

Code below helps to limit period in transit with 4 days from history date. It could have been useful, but in some cases one line may include Stock, Shipped and OOS quantities for History Date, but Transit quantities will be set for distant date (more than 4 days after latest history date).

In [15]:
# df = df[(df['Transit_Date'] >= df['Movement_Date'].min()) & (df['Transit_Date'] <= end_date)]

In [16]:
df = df.reset_index(drop=True)

New dataframe is a grouped version of initial dataframe where all quantity columns are summed up.

In [17]:
df_slim = df.groupby(['Movement_Date','Site_Code','Internal_Code','Item_Name','EAN_Code','Item_Type','Case_Count']) \
                    [['Stock','Transit','Shipped','OOS']].sum()

In [18]:
df_slim = df_slim.reset_index()

Dataframe gets new columns' order.

In [19]:
df_slim = df_slim[['Site_Code','EAN_Code','Item_Type','Case_Count','Movement_Date', \
                   'Stock','Transit','Shipped','OOS','Internal_Code']]

Column names with Movement Types are replaced by Movement Codes that are available at the beginning of the document.

In [20]:
df_slim = df_slim.rename(columns={'Stock':'01','Transit':'04','Shipped':'03','OOS':'05'})

In [21]:
df_melt = pd.melt(df_slim,id_vars=['Site_Code','EAN_Code','Item_Type','Case_Count','Movement_Date','Internal_Code'],
                    value_vars=['01','04','03','05'],
                    var_name='Movement_Code',value_name='Quantity')

Movement Codes are melt for every line.

In [22]:
df_melt = df_melt[['Movement_Code','Site_Code','EAN_Code','Item_Type','Case_Count',
                      'Movement_Date','Quantity','Internal_Code']]

Lines with no Out Of Stock quantity are deleted.

In [23]:
df_melt = df_melt.drop(df_melt[(df_melt['Movement_Code'] == '05') & (df_melt['Quantity'] == 0)].index)

New format for date is applied to Movement Date column. It has no separators and always gets length of 8 symbols.

In [24]:
df_melt["Movement_Date"] = df_melt["Movement_Date"].dt.strftime("%d%m%Y")
df_melt['Movement_Date'] = df_melt['Movement_Date'].astype(str)
df_melt['Movement_Date'] = df_melt['Movement_Date'].str.zfill(8)

Case Count should always have the length of 4 symbols, that's why leading zeros are added.

In [25]:
df_melt['Case_Count'] = df_melt['Case_Count'].str.zfill(4)

Case Count should always have the length of 10 symbols, that's why leading zeros are added.

In [26]:
df_melt['Quantity'] = df_melt['Quantity'].astype(str)
df_melt['Quantity'] = df_melt['Quantity'].str.zfill(10)

Index is reset.

In [27]:
df_melt = df_melt.reset_index(drop=True)

Line below concatenates all the columns without delimeters. Each line corresponds to one movement type per one item on a certain date.

In [29]:
df_invrpt = df_melt.apply(lambda row: ''.join(row.values.astype(str)), axis=1)

print(df_invrpt.head(10))

0    01468032853241246006823701300000202011202000000106151000023046
1    01468032853241246006825754740000202011202000000014001000105264
2    01468032853241246006826634470000202011202000000016461000162982
3    01468032853241246006824074780000202011202000000283601000184411
4    01468032853241246006822084880000202011202000000154201000184415
5    01468032853241246006824834720000202011202000000132001000184418
6    01468032853241246006822004510000242011202000000087171000188046
7    01468032853241246006822514530000242011202000000025921000188100
8    01468032853241246006820074560000242011202000000558531000188102
9    01468032853241246006829915400000242011202000000229771000188105
dtype: object


Line meant for text-file export.

In [30]:
# df_invrpt.to_csv(r'C:\Users\БФедин\OneDrive - GENERIX\InventoryReports\22.11_Report.txt',
#                  sep=' ',index=False,header=False)

Line meant for Excel-file export.

In [31]:
# df.to_excel(r'C:\Users\БФедин\OneDrive - GENERIX\InventoryReports\22.11_Report.xlsx')